In [1]:
%pylab inline
import os.path as op
import sys
import os
import json
import skimage.color
import skimage.filters
import morphsnakes as ms
import io3d.datasets
import scipy.ndimage

pth = op.abspath("..")
sys.path.insert(0, pth)
import scaffan
import scaffan.annotation as scan
import scaffan.image as scim
import scaffan.texture as satex
import scaffan.texture_lbp as salbp
scaffan.image.import_openslide()
import openslide

Populating the interactive namespace from numpy and matplotlib
add path C:\Users\Jirik\Downloads\openslide-win64-20171122\bin


In [2]:
# pth = r"e:data/scaffold/Hamamatsu/PIG-008_P008 LL-P_HE_parenchyme perif..ndpi"
fn = io3d.datasets.join_path("medical", "orig", "CMU-1.ndpi", get_root=True)
anim = scim.AnnotatedImage(fn)


tile_size = 100

2019-07-09 19:03:51.257 | DEBUG    | scaffan.image:__init__:166 - Reading file C:\Users\Jirik/data/medical\orig\CMU-1.ndpi
2019-07-09 19:03:51.281 | DEBUG    | scaffan.image:__init__:180 - Reading the annotation
2019-07-09 19:03:52.900 | DEBUG    | scaffan.annotation:read_annotations:112 - windows annotation output:b'2019-07-09 19:03:52.711 | DEBUG    | scaffan.annotation:ndpa_to_json:64 - PATH: C:\\Users\\Jirik\\Miniconda3\\envs\\scaffan\\Library\\bin;C:\\Users\\Jirik\\Downloads\\openslide-win64-20171122\\bin;C:\\Users\\Jirik\\Downloads\\openslide-win64-20171122\\bin;C:\\Users\\Jirik\\Miniconda3\\envs\\scaffan\\Library\\bin;C:\\Users\\Jirik\\Miniconda3\\envs\\scaffan\\Library\\bin;C:\\Users\\Jirik\\Miniconda3\\envs\\scaffan\\Library\\bin;C:\\Users\\Jirik\\Miniconda3\\envs\\scaffan;C:\\Users\\Jirik\\Miniconda3\\envs\\scaffan\\Library\\mingw-w64\\bin;C:\\Users\\Jirik\\Miniconda3\\envs\\scaffan\\Library\\usr\\bin;C:\\Users\\Jirik\\Miniconda3\\envs\\scaffan\\Library\\bin;C:\\Users\\Jirik\

In [3]:

view1 = anim.get_view_on_annotations("obj1", level=3)

region = view1.get_region_image()
imshow(region)
view1.plot_annotations("obj1")

AttributeError: 'AnnotatedImage' object has no attribute 'get_view_on_annotations'

In [ ]:
view1.annotations


In [ ]:
anim.annotations

In [ ]:
view1.get_annotation_region_raster("obj1")

In [ ]:
view2 = anim.get_view(center=[x, y], level=3, size=[512, 512])
region = view2.get_region_image()
imshow(region)
annotation = view2.annotations[0]
x = np.mean(annotation["x_px"])
y = np.mean(annotation["y_px"])
plot(annotation["region_x_px"], annotation["region_y_px"])

In [ ]:
view1 = anim.get_view_on_annotations("obj1", level=1)
region = view1.get_region_image()
imshow(region)
view1.plot_annotations("obj1")
plt.figure()
mask = view1.get_annotation_region_raster("obj1")
region = view1.get_region_image()
imshow(mask)

In [ ]:
plt.imshow(region)
plt.contour(mask)

In [ ]:
dst = scipy.ndimage.morphology.distance_transform_edt(mask)
plt.imshow(dst)

In [ ]:
middle_pixels = dst > (tile_size/2)
plt.imshow(middle_pixels)

In [ ]:
def nonzero_with_step(data, step):
    print(data.shape)
    datastep = data[::step, ::step]
    print(datastep.shape)
    nzx, nzy = np.nonzero(datastep)
    
    return nzx* step, nzy * step

In [ ]:
nz = nonzero_with_step(middle_pixels, 50 )

In [ ]:
nz

In [ ]:
view1 = anim.get_view_on_annotations("obj1", level=2, show=True)
nz = satex.select_texture_patch_centers_from_one_annotation(anim, "obj1", tile_size=32, step=20)

plt.plot(nz[0], nz[1], "bo")
# nz

In [ ]:
imshow(region)
scan.plot_annotations(anim.annotations, in_region=True)

In [ ]:
annotation

# Get Features

In [ ]:
def show_lbp(lbp):
    n_bins = int(lbp.max() + 1)
    plt.hist(lbp.ravel(), range=(0, n_bins), bins=n_bins, normed=True)
    plt.xlim(xmax=n_points + 2)


In [ ]:
anim.region_location

In [ ]:
def coords_region_px_to_global_px(anim, points):
    px_factor = anim.openslide.level_downsamples[anim.region_level]
    print(px_factor)
    x_px = anim.region_location[0] + points[0] * px_factor
    y_px = anim.region_location[1] + points[1] * px_factor
    
    return x_px, y_px

In [ ]:

nz_base_px = coords_region_px_to_global_px(anim, nz)
points = list(zip(nz_base_px[0], nz_base_px[1]))
print(len(points))

In [ ]:
from skimage.feature import local_binary_pattern


i = 12
anim.set_region(center=points[i], level=0, size=[128, 128])
# anim.set_region(center=anim.region_location, level=4, size=[320, 320])
region_gray = anim.get_region_image(as_gray=True)

# settings for LBP
radius = 3
n_points = 8 * radius
METHOD="uniform"
lbp = local_binary_pattern(region_gray, n_points, radius, METHOD)
lbp.shape
figure()
imshow(region_gray, cmap="gray")
figure()
show_lbp(lbp)

In [ ]:
from skimage.feature import local_binary_pattern

points = list(zip(nz[0], nz[1]))
i = 2

# anim.set_region(center=points[i], level=4, size=[320, 320])
anim.set_region(center=anim.region_location, level=4, size=[320, 320])
region_gray = anim.get_region_image(as_gray=True)

# settings for LBP
radius = 3
n_points = 8 * radius
METHOD="uniform"
lbp = local_binary_pattern(region_gray, n_points, radius, METHOD)
lbp.shape
figure()
imshow(region_gray, cmap="gray")
figure()
show_lbp(lbp)

# Get features from non rectangular area

In [ ]:
anim.set_region_on_annotations("obj1", level=2, show=True)
region_gray = anim.get_region_image(as_gray=True)
mask = anim.get_annotation_region_raster(i="obj1")
plt.figure()
plt.imshow(region_gray)
plt.figure()
plt.imshow(mask)


lbp = local_binary_pattern(region_gray, n_points, radius, METHOD)
len(lbp[mask])

plt.figure()
show_lbp(lbp[mask].ravel())


                                         
                                         